In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your oepn ai api key"

In [ ]:
!pip install -q langchain_community
!pip install -q unstructured
!pip install -q unstructured[pdf]
!pip install -q ragas
!pip install requests

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arxiv 2.1.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


# Загружаем документацию

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

dock_path = ''

loader = DirectoryLoader(doc_path)
documents = loader.load()

# Создаем синтетические данные

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=15, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


embedding nodes:   0%|          | 0/540 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
testset = testset.to_pandas()

# Получаем ответы LLM

In [ ]:
import requests
import pandas as pd
import pandas as pd

# testset = pd.read_csv('/home/dukhanin/dig_break_5-7/datasets/qa_gpt3.csv')


class RagAPP():
    url = "http://localhost:8000/api/management/files"

    def __init__(self):
        pass

    def add_file(self, filename):
        # Replace 'file_path' with the path to the file you want to upload
        try:
            files = {'file': open(filename, 'rb')}
            response = requests.post(self.url, files=files)
            if response.status_code == 200:
                print("File uploaded successfully!")
            else:
                print(f"Failed to upload file with status code: {response.status_code}")
        except Exception as err:
            print(err)
        return requests.get(self.url).json() # All files

    def request(self, question):
        url = 'http://localhost:8000/api/chat/request'  # URL to send the request
        headers = {'Content-Type': 'application/json'}  # Set the Content-Type header
        # Create the request body in JSON format
        payload = {
            "messages": [
                {
                    "content": question,
                    "role": "user"
                }
            ]
        }
        # Send POST request to the API
        response = requests.post(url, json=payload, headers=headers)
        return response.json().get('result').get('content')

# Initialize the RagAPP class
app = RagAPP()

# Example of adding a file through the API
# app.add_file('6.0.0 _ Документация RuStore.pdf')

# Iterate over the dataset, send questions via the API, and store the answers
answers = []
for question in testset['question']:
    answer = app.request(question)
    answers.append(answer)

# Add the answers as a new column to the dataset
testset['answer'] = answers

# Display the updated dataset
print(testset)

# If you want to save the updated dataset to a CSV file
testset.to_csv('updated_testset.csv', index=False)


                                             question  \
0   What is the purpose of setting a minimum map p...   
1   How can markers be combined into clusters usin...   
2   What are the features and benefits of the Trac...   
3   What information does the FURuStorePurchasesRe...   
4   What are the possible values for the "fields" ...   
5   What should you use test cards for when testin...   
6   What are the options available in the Sampling...   
7   How does the GeolocateControl's trackUserLocat...   
8   Which method retrieves tokens for all push not...   
9   What is the role of MapDataSource in MapsSDK i...   
10  How to obtain a user's push token in RuStore p...   
11  How to confirm a purchase in RuStore app for c...   
12  How to upload screenshots and app icon with Ru...   
13  How to integrate RuStore AppUpdate SDK in Unre...   
14  How to confirm purchase of CONSUMABLE products...   
15  Which function in RuStore handles new push not...   

                              

In [ ]:
testset

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,What is the purpose of setting a minimum map p...,[ limited by the above values. In case pan or ...,Setting a minimum map pitch in the context of ...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,Setting a minimum map pitch in the context of ...
1,How can markers be combined into clusters usin...,[ encoded string from the route creation servi...,Markers can be combined into clusters using th...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,To combine markers into clusters using the Map...
2,What are the features and benefits of the Trac...,[ parameter Testing allows to avoid applying i...,The Tracer service provides features such as p...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,The Tracer service offers several features and...
3,What information does the FURuStorePurchasesRe...,[ FString name; UPROPERTY(BlueprintReadOnly) F...,The FURuStorePurchasesResponse structure conta...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,The FURuStorePurchasesResponse structure conta...
4,"What are the possible values for the ""fields"" ...","[ ""125167""\n\n616\n\ngeometry field descriptio...","fieldname1, fieldname2, ....fieldnameN",simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,"The possible values for the ""fields"" parameter..."
5,What should you use test cards for when testin...,[. You can also test purchases if an app has a...,"When testing in-app purchases, you should use ...",simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,"When testing in-app purchases, you should use ..."
6,What are the options available in the Sampling...,[Below are the SamplingProfilerConfiguration.B...,The options available in the SamplingProfilerC...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,The `SamplingProfilerConfiguration.Builder` pr...
7,How does the GeolocateControl's trackUserLocat...,[DefaultPosition will be used as the control's...,The trackUserLocation option in GeolocateContr...,simple,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,The `trackUserLocation` option in the `Geoloca...
8,Which method retrieves tokens for all push not...,[ below to work with HMS: ● add to build.gradl...,Call the getTokens() method to retrieve tokens...,reasoning,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,The method to retrieve tokens for all push not...
9,What is the role of MapDataSource in MapsSDK i...,[ encoded string from the route creation servi...,MapDataSource in MapsSDK integration allows fo...,reasoning,[{'source': '/home/dukhanin/dig_break_5-7/docs...,True,"In the context of MapsSDK integration, the `Ma..."


# Делаем evaluate

In [ ]:
import pandas as pd

# Assuming testset is your DataFrame
testset_dict = testset.to_dict(orient='list')

# Print the dictionary to check
print(testset_dict)


{'question': ['What is the purpose of setting a minimum map pitch in the context of map display options?', 'How can markers be combined into clusters using the MapsSDK?', 'What are the features and benefits of the Tracer service for mobile app developers?', 'What information does the FURuStorePurchasesResponse structure contain?', 'What are the possible values for the "fields" parameter in the general request parameters for the geocoding service?', 'What should you use test cards for when testing in-app purchases?', 'What are the options available in the SamplingProfilerConfiguration.Builder for profiling configuration?', "How does the GeolocateControl's trackUserLocation option affect the behavior of the control?", 'Which method retrieves tokens for all push notification providers in the HMS integration process?', 'What is the role of MapDataSource in MapsSDK integration, with its ability to add sources and layers in one call and cluster markers?', "How to obtain a user's push token i

In [ ]:
testset_dict

In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict(testset_dict)

In [ ]:
dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 16
})

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
ragas_vertexai_llm = LangchainLLMWrapper(generator_llm)

result = evaluate(
    dataset,
    llm=generator_llm,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result



Evaluating:   0%|          | 0/64 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 0.8046, 'answer_relevancy': 0.8982, 'context_recall': 0.9875}